In [1]:
%matplotlib inline
import matplotlib
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import scikits.bootstrap as boot
import matplotlib.pyplot as plt
import re
import networkx as nx
from wordcloud import WordCloud
from textblob import TextBlob
from math import isnan
import statsmodels.api as sm
from scipy.stats import mannwhitneyu
import scikits.bootstrap as boot

pd.set_option('display.max_columns', None)

# view all columns
pd.set_option('display.max_columns', None)
plt.rcParams['ps.useafm'] = True
plt.rcParams['font.sans-serif'] = 'Spica Neue'
#plt.rcParams['font.sans-serif'] = 'IPAexGothic'

In [ ]:
####################
# network analysis #
####################

In [ ]:
# import file path
def dates2fpaths(pkl_dir, start_date_str, end_dates_str):
    start_date = pd.to_datetime(start_date_str).tz_localize(None)
    end_date = pd.to_datetime(end_dates_str).tz_localize(None)
    diff = (end_date - start_date).days + 1
    dt_list = [start_date + timedelta(days=i) for i in range(diff)]
    return [pkl_dir.joinpath(str(dt)[0:10].replace('-','')+'.pkl.xz') for dt in dt_list]

# capture statistic information of vaccine texts
def dataset_summary_vaccine(pkl_dir, fpaths):
    # initialize variables
    tweet_num =0
    retweet_num =0
    reply_num =0
    favorite_count =0
    user_name = []
    user_hashtag = []
    
    for f in fpaths:
        try:
            dt = pd.to_datetime(str(f.name)[0:8])
            print(dt)
            df = pd.read_pickle(f)

            # English only
            df = df[(df['lang'] == 'en')] 
            df = df[df['text'].str.contains('vaccines|vaccine|vaccinated|vaccination|vaccineoutside|vaccinate|vaccinologist|vaccinert|coronavirusvaccine', na=False)]
            
            # number of retweet
            day_retweet = len(df[df['retweeted_status_created_at'].notnull()])
            retweet_num = retweet_num + day_retweet
            
            # number of reply
            day_reply = len(df[df['in_reply_to_user_id_str'].notnull()])  
            reply_num = reply_num + day_reply           
            
            # number of tweet
            tweet_num = tweet_num + len(df)- day_retweet - day_reply
            
            # number of favourite
            for i in range(len(df['favorite_count'])):
                favorite_count += df['favorite_count'][i]
            
            # all accounts
            for i in range(len(df['user_screen_name'])):
                user_name.append(df['user_screen_name'][i])
            
            # all hashtags
            for i in df['hashtags']:
                # tokenize
                tokenizer = nltk.RegexpTokenizer(r"\w+")
                new_words = tokenizer.tokenize(i)
                for j in new_words:
                    user_hashtag.append(j)
                    
        except:
            print('error: ', f)
    return tweet_num, retweet_num, reply_num, favorite_count, user_name, user_hashtag

In [ ]:
# retweet network
# contruct retweet network
def retweet_netwroks(pkl_dir, fpaths, filename):

    e = []
    for f in fpaths:
        try:
            dt = pd.to_datetime(str(f.name)[0:8])
            print(dt)
            df = pd.read_pickle(f)

            # English only
            df = df[(df['lang'] == 'en')] 
            
            # retweets only
            df = df[df['retweeted_status_created_at'].notnull()]

            # find keyword cocurrence        
            df = df[df['text'].str.contains('vaccines|vaccine|vaccinated|vaccination|vaccineoutside|vaccinate|vaccinologist|vaccinert|coronavirusvaccine', na=False)]
            
            # retweet edges
            # from (source) and to (target or user who post the original)
            edge_dic = df.groupby(by=['user_screen_name', 'retweeted_status_screen_name']).size().to_dict()
            for k, v in edge_dic.items():
                e.append([k[0], k[1], {'weight': v}])
        except:
            print('error: ', f)
        
    D = nx.from_edgelist(e, create_using=nx.DiGraph)
    nx.write_gexf(D, filename)

In [ ]:
# Use Louvain algorithm in Gephi to find Pro and Anti clusters.
# Download nodes information of each cluster by Gephi named "RTnet_vacc_2021_06_30>20.csv" 
# which modularity_class of '4' represent for Pro and '0' represent for Anti. 

In [ ]:
# Network measures(including basic information of network, Network density, Clustering coefficient and Average distance)
def retweet_network_information(name):
    # RT network
    G = nx.read_gexf(name)
    print('All nodes:', nx.info(G))
    print('\n')

    # RT network >= 20
    node = list(G.nodes)
    for i in range(G.number_of_nodes()):
        if G.degree[node[i]] < 20:
            G.remove_node(node[i])
    print('node degree>=20:', nx.info(G))
    
    # save gexf
    # nx.write_gexf(G, 'RTnet_vacc_2020_12_01 - 2021_06_30>20.gexf')

    # nodes.csv from Gephi
    nodes = pd.read_csv('RTnet_vacc_2021_06_30>20.csv')

    # read nodes for Anti and Pro
    # 4 represent for Pro, 0 represent for Anti 
    pro_nodes = list(nodes[nodes['modularity_class']==4]['Id'])
    anti_nodes = list(nodes[nodes['modularity_class']==0]['Id'])

    print('\n')
    print('number of Pro: ', len(pro_nodes))
    print('number of Anti: ',len(anti_nodes))
    print('\n')

    # fliter nodes for pro
    G_pro = G.subgraph(pro_nodes)
    print(nx.info(G_pro))

    print('\n')
    # fliter nodes for anti
    G_anti = G.subgraph(anti_nodes)
    print(nx.info(G_anti))

    print('\n')
    # network density
    G_network_density = nx.density(G)
    print('G_network_density: ', G_network_density)

    G_pro_network_density = nx.density(G_pro)
    print('G_pro_network_density: ', G_pro_network_density)

    G_anti_network_density = nx.density(G_anti)
    print('G_anti_network_density: ', G_anti_network_density)
    print('\n')
    
    # global clustering coeffcient

    G_average_clustering = nx.average_clustering(G)
    G_pro_average_clustering = nx.average_clustering(G_pro)
    G_anti_average_clustering = nx.average_clustering(G_anti)

    print('average local clustering coefficient of entirety: ', G_average_clustering)
    print('average local clustering coefficient of Pro: ', G_pro_average_clustering)
    print('average local clustering coefficient of Anti: ', G_anti_average_clustering)
    print('\n')
    
    # average distance
    G_pro_average_distance = nx.average_shortest_path_length(nx.to_undirected(G_pro))
    G_anti_average_distance = nx.average_shortest_path_length(nx.to_undirected(G_anti))

    print('Average distance of Pro: ', G_pro_average_distance)
    print('Average distance of Anti: ', G_anti_average_distance)

In [ ]:
# Find Top-10 users 
def find_top_users(name):
    G = nx.read_gexf(name)
    a = G.degree
    degree = sorted(G.degree(), key=lambda x:x[1], reverse=True)

    def find_top_users_for_cluster(number):

        node  = list(nodes[nodes['modularity_class']==number]['Id'])
        num = len(list(nodes[nodes['modularity_class']==number]['Id']))

        # names and degrees of all nodes labeled with 'number'
        degree0  = []
        for i in range(len(degree)):
            if degree[i][0] in node:
                degree0.append(degree[i])
        ratio = num/len(nodes)*100
        ratio=('%.2f' % ratio)

        # 打印top 10 
        top_10 = sorted(degree0, key=lambda x:x[1], reverse=True)[:10]
        print("Top 10 of {}:".format(number))
        # print('Ratio:{}%'.format(ratio))
        print(top_10)  

    # print ratio and top user of each cluster
    cluster_name = [4,0]
    for i in cluster_name:
        find_top_users_for_cluster(i)
        print('---------------------')

In [5]:
# revising...
def Distribution_of_the_indegree(G):
    
    # RT network
    G = nx.read_gexf(name)

    # nodes.csv from Gephi
    nodes = pd.read_csv('RTnet_vacc_2021_06_30>20.csv')

    # read nodes for Anti and Pro
    # 4 represent for Pro, 0 represent for Anti 
    pro_nodes = list(nodes[nodes['modularity_class']==4]['Id'])
    anti_nodes = list(nodes[nodes['modularity_class']==0]['Id'])
    
    degree_hist = np.array(nx.degree_histogram(G))
    pk = degree_hist / degree_hist.sum()
    plt.xlabel('In Degree of G')
    plt.ylabel('Fraction of Nodes')
    plt.loglog(pk)
    #plt.xlim(1, 11000)

In [ ]:
pkl_dir = Path('CoronaTweets_pkl_20210630')
#fpaths = sorted(pkl_dir.glob('*.pkl.xz'))
fpaths = dates2fpaths(pkl_dir, '2020-02-20', '2021-06-30')
# capture statistic information of vaccine
tweet_num_vaccine, retweet_num_vaccine, reply_num_vaccine, favorite_count_vaccine, user_name_vaccine, user_hashtag_vaccine = dataset_summary_vaccine(pkl_dir, fpaths)
print(tweet_num_vaccine, retweet_num_vaccine, reply_num_vaccine, favorite_count_vaccine, len(set(user_name_vaccine)), len(set(user_hashtag_vaccine)))

In [ ]:
# save retweet network

# contruct retweet network
pkl_dir = Path('CoronaTweets_pkl_20210630')
# fpaths = sorted(pkl_dir.glob('*.pkl.xz'))
fpaths = dates2fpaths(pkl_dir, '2020-02-20', '2021-06-30')
retweet_netwroks(pkl_dir, fpaths, 'RTnet_vacc_2021_06_30>20.gexf')

# contruct retweet network before mass vaccination
pkl_dir = Path('CoronaTweets_pkl_20210630')
#fpaths = sorted(pkl_dir.glob('*.pkl.xz'))
fpaths = dates2fpaths(pkl_dir, '2020-02-20', '2020-12-01')
retweet_netwroks(pkl_dir, fpaths, 'RTnet_vacc_2020_02_20 - 2020_12_01.gexf')

# contruct retweet network after mass vaccination
pkl_dir = Path('CoronaTweets_pkl_20210630')
#fpaths = sorted(pkl_dir.glob('*.pkl.xz'))
fpaths = dates2fpaths(pkl_dir, '2020-12-02', '2021-06-30')
retweet_netwroks(pkl_dir, fpaths, 'RTnet_vacc_2020_12_01 - 2021_06_30.gexf')

In [ ]:
# Measures and find Top-10 users for different retweet network which 2020_12_01 represent the timepoint for mass vaccination 
# 2020_02_20 To 2021_06_30
print('Timeline: From 2020_02_20 To 2021_06_30')
retweet_network_information('RTnet_vacc_2021_06_30.gexf')
find_top_users('RTnet_vacc_2021_06_30>20.gexf')

# 2020_02_20 To 2020_12_01
print('Timeline: From 2020_02_20 To 2020_12_01')
retweet_network_information('RTnet_vacc_2020_02_20 - 2020_12_01.gexf')
find_top_users('RTnet_vacc_2020_02_20 - 2020_12_01>20.gexf')

# 2020_12_01 To 2021_06_30
print('Timeline: From 2020_12_01 To 2021_06_30')
retweet_network_information('RTnet_vacc_2020_12_01 - 2021_06_30.gexf')
find_top_users('RTnet_vacc_2020_12_01 - 2021_06_30>20.gexf')

In [ ]:
# revising...
# Distribution of the indegree
Distribution_of_the_indegree('RTnet_vacc_2021_06_30.gexf')

In [ ]:
#######################
# linguistic analysis #
#######################

In [ ]:
# collect texts
def collect_texts(pkl_dir, fpaths, label):
    if label == 'tweets':
        df_pro = []
        df_anti = []
        for f in fpaths:
            try:

                # read excel of anti and pro nodes
                nodes = pd.read_csv('RTnet_vacc_2021_06_30>20.csv')
                nodes_of_pro= list(nodes[nodes['modularity_class']==4]['Id'])
                nodes_of_anti = list(nodes[nodes['modularity_class']==0]['Id'])

                dt = pd.to_datetime(str(f.name)[0:8])
                print(dt)
                df = pd.read_pickle(f)

                # English only
                df = df[(df['lang'] == 'en')]

                # retweets only
                # df = df[df['retweeted_status_created_at'].notnull()]

                # replies only
                # df = df[df['in_reply_to_user_id_str'].notnull()]

                # tweets only
                df = df[df['retweeted_status_created_at'].isnull()]
                df = df[df['in_reply_to_user_id_str'].isnull()]

                # find tweets containing keywords      
                df = df[df['text'].str.contains('vaccines|vaccine|vaccinated|vaccination|vaccineoutside|vaccinate|vaccinologist|vaccinert|coronavirusvaccine', na=False)]

                # save only time and text
                # df = df[['created_at', 'text']]

                # save dataframe
                for index, row in df.iterrows():
                    # sort texts for anti and pro
                    if row['user_screen_name'] in nodes_of_anti:
                        df_anti.append(row[['created_at', 'text']])
                    elif row['user_screen_name'] in nodes_of_pro:
                        df_pro.append(row[['created_at', 'text']])
            except:
                print('error: ', f)
        return df_pro, df_anti
    elif label == 'replies':
        df_pro = []
        df_anti = []
        for f in fpaths:
            try:

                # read excel of anti and pro nodes
                nodes = pd.read_csv('RTnet_vacc_2021_06_30>20.csv')
                nodes_of_pro= list(nodes[nodes['modularity_class']==4]['Id'])
                nodes_of_anti = list(nodes[nodes['modularity_class']==0]['Id'])

                dt = pd.to_datetime(str(f.name)[0:8])
                print(dt)
                df = pd.read_pickle(f)

                # English only
                df = df[(df['lang'] == 'en')]

                # retweets only
                # df = df[df['retweeted_status_created_at'].notnull()]

                # replies only
                df = df[df['in_reply_to_user_id_str'].notnull()]

                # tweets only
                # df = df[df['retweeted_status_created_at'].isnull()]
                # df = df[df['in_reply_to_user_id_str'].isnull()]

                # find tweets containing keywords      
                df = df[df['text'].str.contains('vaccines|vaccine|vaccinated|vaccination|vaccineoutside|vaccinate|vaccinologist|vaccinert|coronavirusvaccine', na=False)]

                # save only time and text
                # df = df[['created_at', 'text']]

                # save dataframe
                for index, row in df.iterrows():
                    # sort texts for anti and pro
                    if row['user_screen_name'] in nodes_of_anti:
                        df_anti.append(row[['created_at', 'text']])
                    elif row['user_screen_name'] in nodes_of_pro:
                        df_pro.append(row[['created_at', 'text']])
            except:
                print('error: ', f)
        return df_pro, df_anti

In [ ]:
pkl_dir = Path('CoronaTweets_pkl_20210630')
#fpaths = sorted(pkl_dir.glob('*.pkl.xz'))
fpaths = dates2fpaths(pkl_dir, '2020-02-20', '2021-06-30')
# collect teeets
tweet_pro, tweet_anti = collect_tweets(pkl_dir, fpaths, 'tweets')
# collect replies
reply_pro, reply_anti = collect_replies(pkl_dir, fpaths, 'replies')

# save texts
tweet_pro = pd.DataFrame(tweet_pro)
tweet_pro.to_csv('tweet_pro')

tweet_anti = pd.DataFrame(tweet_anti)
tweet_anti.to_csv('tweet_anti')

reply_pro = pd.DataFrame(reply_pro)
reply_pro.to_csv('reply_pro')

reply_anti = pd.DataFrame(reply_anti)
reply_anti.to_csv('reply_anti')

In [ ]:
# Then we use LIWC to analyze texts of each group. LIWC can help us get kinds(e.g. negative, positive, analytic) of scores of each text.
# We use text's score in each gruop to find if there exist significance difference by independent t-test.
# And we conclude each group scores in files 'LIWC2015 Results (tweet_pro.csv).csv', 'LIWC2015 Results (tweet_anti.csv).csv'
# 'LIWC2015 Results (reply_pro.csv).csv', 'LIWC2015 Results (reply_anti.csv).csv'

In [ ]:
# concatenate scores results for 'tweet'
tweet_pro = pd.read_csv('LIWC2015 Results (tweet_pro.csv).csv')
tweet_anti = pd.read_csv('LIWC2015 Results (tweet_anti.csv).csv')
tweet_vaccine = tweet_pro.append(tweet_anti).dropna() 

In [ ]:
# concatenate scores results for 'replies'
reply_pro = pd.read_csv('LIWC2015 Results (reply_pro.csv).csv')
reply_anti = pd.read_csv('LIWC2015 Results (reply_anti.csv).csv')
reply_vaccine = reply_pro.append(reply_anti).dropna() 

In [ ]:
def t_test(df1, df2, column):
    if df1[column].agg(np.std) != df2[column].agg(np.std): 
        
        df1_column = sm.stats.DescrStatsW(df1[column])
        df2_column = sm.stats.DescrStatsW(df2[column])
        
        mean = df2_column.mean - df1_column.mean
        pvalue = list(sm.stats.CompareMeans(df2_column, df1_column).ttest_ind(usevar='unequal'))[1]
        result.append([mean, pvalue])

    else:
        print("Pooled")

In [ ]:
# We want to check the significance difference of means in anti and pro in reply texts and tweet texts respectively

# Tweets
result = []
# categories we want to compare within LIWC
categories = ['affect', 'posemo', 'negemo', 'Analytic', 'funct', 'pronoun']
for category in categories:
    t_test(tweet_pro, tweet_anti, category)

# print p-value for each category    
k = 0
for i in categories:
    print(i)
    print('p-value: ',result[k][1])
    k=k+1 

# confidence intervals with bootstrap
for category in categories:
    print(boot.ci(tweet_pro[category]))
    print(boot.ci(tweet_anti[category]))
    

# Replies
result = []
# categories we want to compare within LIWC
categories = ['affect', 'posemo', 'negemo', 'Analytic', 'funct', 'pronoun']
for category in categories:
    t_test(reply_pro, reply_anti, category)

# print p-value for each category    
k = 0
for i in categories:
    print(i)
    print('p-value: ',result[k][1])
    k=k+1
    
# confidence intervals with bootstrap
for category in categories:
    print(boot.ci(reply_pro[category]))
    print(boot.ci(reply_anti[category]))

In [ ]:
# We also want to check if there are significance difference exist in the mode of reply and tweet texts with negative emotion
# before and after mass vaccination.

# We find the timepoint of 2020-12-01 mannualy and divide all the texts

tweet_pro_before = tweet_pro[0:77608]
tweet_pro_after = tweet_pro[77608:]

tweet_anti_before = tweet_anti[0:25983]
tweet_anti_after = tweet_anti[25983:]

reply_pro_before = reply_pro[0:7945]
reply_pro_after = reply_pro[7945:]

reply_anti_before = reply_anti[0:12888]
reply_anti_after = reply_anti[12888:]

# And we compare all the groups by negetive emotion before and after 2020-12-01

# tweet_pro_before VS tweet_anti_before
t_test(tweet_pro_before, tweet_anti_before, 'negemo')

# tweet_pro_after VS tweet_anti_after
t_test(tweet_pro_after, tweet_anti_after, 'negemo')

# reply_pro_before VS reply_anti_before
t_test(reply_pro_before, reply_anti_before, 'negemo')

# tweet_pro_after VS tweet_anti_after
t_test(reply_pro_after, reply_anti_after, 'negemo')

In [ ]:
# Merge the texts of pro and anti by Tweet and Reply
pro = tweet_pro.append(reply_pro)
anti = tweet_anti.append(reply_anti)

# compare moral perspectives in Reply and Anti
moral_name = ['HarmVirtue', 
       'HarmVice',
       'FairnessVirtue',
       'FairnessVice',
       'IngroupVirtue',
       'IngroupVice',
       'AuthorityVirtue',
       'AuthorityVice',
       'PurityVirtue',
       'PurityVice',
       'MoralityGeneral']

# t-test
result = []
for moral in moral_name:
    t_test(pro, anti, moral)
    
# print p-value for each category    
k = 0
for i in moral_name:
    print(i)
    print('p-value: ',result[k][1])
    k=k+1